# 1 Dependencies

In [3]:
from datetime import date
from typing import Optional, List
from sqlalchemy import create_engine, text, Integer, String, ForeignKey, Date, CheckConstraint
from sqlalchemy.engine import URL
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session
from sqlalchemy.exc import ProgrammingError

# 2 Establish a connection

In [ ]:
server_name = "Alstation"
username = "sa"
password = "D@tabases"

connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server_name};UID={username};PWD={password}"
url_string = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(url_string)

# 3 Create a database

In [ ]:
database_name = "bookstore"

try:
    with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
        conn.execute(text(f"CREATE DATABASE {database_name}"))
except ProgrammingError as err:
    err_string = str(err)
    if "'bookstore' already exists" in err_string:
        print(f"A database named '{database_name}' already exists.")
    else:
        print(err)

A database named 'bookstore' already exists.


# 4 Create tables

## 4.1 Required

In [ ]:
class Base(DeclarativeBase):
    pass

class Authors(Base):
    __tablename__ = "authors"

    id: Mapped[int] = mapped_column(primary_key=True)
    first_name: Mapped[str] = mapped_column(String(30))
    last_name: Mapped[str] = mapped_column(String(30))
    birth_date: Mapped[Optional[date]] = mapped_column(Date)

    # deleting an author deletes all their books
    books: Mapped[List["Books"]] = relationship(back_populates="authors", cascade="all, delete-orphan")

class Books(Base):
    __tablename__ = "books"
    __table_args__ = (CheckConstraint("price > 0", name="check_price_positive"))  # what if I want several?

    isbn: Mapped[int] = mapped_column(primary_key=True)
    title: Mapped[str] = mapped_column(String(100))
    language: Mapped[Optional[str]] = mapped_column(String(30))
    price: Mapped[int] = mapped_column(Integer)
    publication_date: Mapped[Optional[date]] = mapped_column(Date)
    author_id: Mapped[int] = mapped_column(ForeignKey("authors.id"))

    authors: Mapped["Authors"] = relationship(back_populates="books")
    # deleting a book deletes it's inventory
    inventory: Mapped[List["Inventory"]] = relationship(back_populates="books", cascade="all, delete-orphan")

class Stores(Base):
    __tablename__ = "stores"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(50))
    address: Mapped[Optional[str]] = mapped_column(String(100))

    # deleting a store deletes it's inventory
    inventory: Mapped[List["Inventory"]] = relationship(back_populates="stores", cascade="all, delete-orphan")

class Inventory(Base):
    __tablename__ = "inventory"

    store_id: Mapped[int] = mapped_column(ForeignKey("stores.id"), primary_key=True)
    isbn: Mapped[int] = mapped_column(ForeignKey("books.isbn"), primary_key=True)
    amount: Mapped[int] = mapped_column(Integer, default=0)

    books: Mapped["Books"] = relationship(back_populates="inventory")
    stores: Mapped["Stores"] = relationship(back_populates="inventory")

## 4.2 Additional

In [ ]:
# Do I need to specify int and Integer for PK and FK? 
# How do I do nvarchar(max) or do I even need to? Should I not even specify string length?
# how would cascades work here? 
# more checks and defaults? 


class BookAuthors(Base):
    __tablename__ = "book_authors"

    isbn: Mapped[int] = mapped_column(ForeignKey("books.isbn"))
    author_id: Mapped[int] = mapped_column(ForeignKey("authors.id"))
    role: Mapped[str] = mapped_column(String(20))

    books: Mapped["Books"] = relationship(back_populates="bookauthors")
    authors: Mapped["Authors"] = relationship(back_populates="bookauthors")

class Customers(Base):
    __tablename__ = "customers"
    # check for email containing % + @ + %

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    first_name: Mapped[str] = mapped_column(String(30))
    last_name: Mapped[str] = mapped_column(String(30))
    email: Mapped[str] = mapped_column(String(80))
    membership_level: Mapped[str] = mapped_column(String(8))

class Orders(Base):
    __tablename__ = "orders"

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    customer_id: Mapped[int] = mapped_column(ForeignKey("customers.id"))
    order_date: Mapped[date] = mapped_column(Date)

    customers: Mapped["Customers"] = relationship(back_populates="orders")

class OrderDetails(Base):
    __tablename__ = "order_details"

    order_id: Mapped[int] = mapped_column(ForeignKey("orders.id"))
    book_id: Mapped[int] = mapped_column(ForeignKey("books.isbn"))
    quantity: Mapped[int] = mapped_column(Integer)

    orders: Mapped["Orders"] = relationship(back_populates="order_details")
    books: Mapped["Books"] = relationship(back_populates="order_details")

class Reviews(Base):
    __tablename__ = "reviews"

    isbn: Mapped[int] = mapped_column(ForeignKey("books.isbn"))
    customer_id: Mapped[int] = mapped_column(ForeignKey("customers.id"))
    rating: Mapped[int] = mapped_column(Integer)

    books: Mapped["Books"] = relationship(back_populates="reviews")
    customers: Mapped["Customers"] = relationship(back_populates="reviews")

## 4.3 Creation